In [1]:
# 케라스 API 이용해서 패션 MNIST 데이터셋 다운
from tensorflow import keras

(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

# 데이터 전처리: 표준화, 2차원 배열 1차원으로 펼치기
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28 * 28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


## 2개의 층
7-1장에서는 하나의 층으로된 신경망 모델을 만들었는데, 그 구조는 입력층 뉴런 784개, 출력층 뉴런 10개, 소프트맥스 함수이다. 여기에서는 층을 하나 더 만들어 입력층 뉴런 784개, 중간층 뉴런 100개, 활성화 함수, 출력층 뉴런 10개, 소프트맥스 함수 구조로 만들어보자. 여기서 입력층과 출력층 중간에 있는 모든 층을 **은닉층(hidden layer)**이라 한다. 은닉층과 출력층은 뉴런 이후에 활성화 함수를 거쳐야 하는데, 출력층은 이진 분류일 때는 시그모이드, 다중 분류일 때는 소프트맥스 함수를 사용하는 것처럼 사용 가능한 종류가 제한되어있으나 은닉층에서는 사용 가능 종류가 비교적 자유롭다. 대표적으로는 시그모이드 함수와 렐루(ReLU) 함수를 이용한다

> 참고
- 회귀를 위한 신경망의 출력층에서는 활성화 함수를 사용하지 않는다. 분류에서는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용하지만 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용하지 않고 출력층의 선형 방정식의 계산을 그대로 출력한다. 회귀를 사용하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않으면 된다

은닉층에 활성화 함수를 적용하는 이유는 다음과 같다. 먼저 다음 식을 보자
$$a \times 4 + 2 = b$$
$$b \times 3 - 5 = c$$
$$a \times 12 + 1 = c$$
두 번째 식에 첫 번째 식을 대입하면 세 번째 식이 나온다. 이렇게 되면 굳이 b를 만들 필요 없이 바로 세 번째 식을 사용하도 무방하다. 이런 것처럼 선형 방정식으로 계산한 은닉층의 계산값을 바로 내놓으면 은닉층이 없는 것과 비슷한 결과를 내기 때문에 다음처럼 비선형 계산을 넣어준다
$$a \times 4 + 2 = b$$
$$log(b) = k$$
$$k \times 3 - 5 = c$$
이처럼 비선형 계산을 넣어주면 다음 층의 계산과 합쳐지지 않고 나름의 역할을 수행할 수 있게 된다

In [2]:
# 은닉층의 활성화 함수를 시그모이드로
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

# 심층 신경망
model = keras.Sequential([dense1, dense2])

# 층에 대한 정보
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층 마다 층 이름(클래스) | 출력 크기 | 파라미터 개수가 나온다. 첫 번째 층을 보면 출력 크기가 (None, 100)으로 되어있는데, None은 샘플의 개수를 나타낸다. 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 미니배치 경사 하강법을 이용하게 된다. 기본 배치사이즈는 32개지만 fit() 메서드의 batch_size로 크기를 조정할 수 있기 때문에 어떤 크기에도 유연하게 대응할 수 있도록 None으로 설정되었다. 파라미터의 개수는 뉴런을 잇는 모든 간선 수 + 간선의 화살표 방향에 있는 뉴런의 개수만큼의 절편이다. 따라서 첫 번째 층의 파라미터의 수는 784 * 100 + 100 = 78,500개가 된다. 아래의 Non-trainable params는 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 생길 때 그 층의 파라미터 개수가 표시된다

## 층을 추가하는 다른 방법


In [3]:
# 위에서 사용했던 방법
model = keras.Sequential([
        keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),
                           name='hidden'),
        keras.layers.Dense(10, activation='softmax', name='output')
        ], name='패션 MINST 모델')

model.summary()

Model: "패션 MINST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 많은 층을 만들 때 편리한 add() 메서드를 활용한 층 추가
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

# 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5650 - accuracy: 0.8080
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.4086 - accuracy: 0.8535
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3745 - accuracy: 0.8651
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3511 - accuracy: 0.8734
Epoch 5/5
1500/1500 [=====================

## 렐루 함수
시그모이드는 양 끝으로 갈수록 기울기가 0이기 때문에, 계속 중첩될 시 기울기 소실(gradient vanishing)이라 불리는 문제를 일으켜 학습을 방해한다. 따라서 이를 해결하기 위해 **렐루 함수(ReLU)**를 만들었는데, 이는 다음과 같다
$$f(x)=
\begin{cases}
0,\;if\;x<0\\
x,\;if\;x\geq0
\end{cases}$$
$$ i.e. f(x)=max(0, x) $$

케라스에서 제공하는 편리한 층이 있는데, Flatten 층은 reshape로 이미지를 1차원으로 펼쳤던 수고를 덜어준다. 물론 인공 신경망의 성능을 위해 기여하는 바는 없으나 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라 부른다

In [5]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 훈련
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5333 - accuracy: 0.8120
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3935 - accuracy: 0.8579
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3563 - accuracy: 0.8721
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3354 - accuracy: 0.8792
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3180 - accuracy: 0.8862


In [7]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3576 - accuracy: 0.8757


[0.3576275110244751, 0.8756666779518127]

## 옵티마이저
compile() 메서드에서는 일반적으로 케라스 기본 경사 하강법 알고리즘인 RMSprop을 사용한다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하는데 이들을 **옵티마이저(optimizer)**라고 한다

기본적인 옵티마이저로는 SGD가 있다. 이름은 SGD지만 1개의 샘플을 뽑아 훈련하지 않고 미니배치를 사용한다. SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 메서드를 'sgd'로 지정한다
``` python
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',
              metrics='accuracy')
```
이는 아래 코드와 동일하다
``` python
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',
              metrics='accuracy')
```
SGD의 학습률을 바꾸고싶으면 다음과 같이 하면 된다. 기본값은 0.01이다
``` python
sgd = keras.optimizers.SGD(learning_rate=0.1)
```

기본 경사 하강법 옵티마이저는 모두 SGD클래스에서 제공한다. SGD 클래스의 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 이전의 그레이디언트를 가속도처럼 사용하는 **모멘텀 최적화(momentum optimization)**를 사용한다. 보통 momentum 매개변수는 0.9 이상을 지정한다. 그리고 다음처럼 SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 **네스테로프 모멘텀 최적화(nesterov momentum optimization)**를 사용한다
``` python
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
```

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 최적점에 안정적으로 수렴할 수 있다. 이런 학습률을 **적응적 학습률(adaptive learning rate)**이라 한다. 적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSprop이다. 각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정할 수 있다. optimizer 매개변수의 기본값이 'rmsprop'이다. 두 옵티마이저의 매개변수를 바꾸고 싶다면 Adagrad와 RMSprop 클래스 객체를 만들어 수정하면 된다.
``` python
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy',
              metrics='accuracy')
```
``` python
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy',
              metrics='accuracy')
```

모멘텀 최적화와 RMSprop의 장점을 접목하여 만든 옵티마이저인 Adam도 있다. 이 3개의 클래스는 학습률 기본값으로 0.001을 사용한다

In [8]:
# Adam으로 모델 훈련
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5301 - accuracy: 0.8164
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3946 - accuracy: 0.8568
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3541 - accuracy: 0.8708
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3272 - accuracy: 0.8804
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3102 - accuracy: 0.8859


In [9]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3373 - accuracy: 0.8781


[0.3373139500617981, 0.878083348274231]